In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install rasterio

%cd "/content/drive/MyDrive/img-label-correction-SAM/SAM/segment-anything"
!pip install .

import torch
import rasterio
import numpy as np
import pandas as pd
from datetime import datetime
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon
from PIL import Image
from rasterio.warp import transform
from rasterio.features import rasterize, shapes
from segment_anything import sam_model_registry, SamPredictor

import logging
logging.getLogger('rasterio._env').setLevel(logging.ERROR)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 86.9 MB/s eta 0:00:00
/content/drive/MyDrive/img-label-correction-SAM/SAM/segment-anything
Processing /content/drive/MyDrive/img-label-correction-SAM/SAM/segment-anything
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=d5223a3b908bbc7d9a1caba46c622f260e0a36e16ed272ff7afacef6eae4bbbf
  Stored in directory: /root/.cache/pip/wheels/28/6a/fc/2f204cbca92c68e6964efccdf30bba56b0f77ae04d580673e5
Successfully built segment_anything
cuda


In [2]:
#valid_uid_list=['95d5860f-c615-5903-833c-bbcb2a70ab5e','695c921c-0b42-5e3b-a88a-23924816bc08']

prompt='mask'
basemodel='sam'

data_folder_path = '/content/drive/MyDrive/img-label-correction-SAM/data/raw/'
good_geojson_path = data_folder_path+'GroundTruth_Final_20250625.geojson'
bad_geojson_path = data_folder_path+'Starting_Polygons_20250625.geojson'

output_folder_path = '/content/drive/MyDrive/img-label-correction-SAM/output/'
output_images_folder_path = output_folder_path+'images/'
output_metadata_folder_path = output_folder_path+'metadata/'

geojson_output_path = output_metadata_folder_path + "predicted_polygons_"+basemodel+"_"+prompt+"_prompt.geojson"
manifest_path = output_metadata_folder_path+"rts_auto_segmentation_manifest.csv"
missing_uid_list_path = output_metadata_folder_path+'missing_uid_list.txt'

sam_folder_path = '/content/drive/MyDrive/img-label-correction-SAM/SAM/'
models = [
  {
    "basemodel": "sam",
    "name": "sam_base",
    "config": "vit_b",
    "checkpoint": sam_folder_path+"sam_vit_b_01ec64.pth"
  },
  {
    "basemodel": "sam",
    "name": "sam_large",
    "config": "vit_l",
    "checkpoint": sam_folder_path+"sam_vit_l_0b3195.pth"
  },
  {
    "basemodel": "sam",
    "name": "sam_huge",
    "config": "vit_h",
    "checkpoint": sam_folder_path+"sam_vit_h_4b8939.pth"
  }
]

polygon_records = []
for model_info in models:
  basemodel = model_info["basemodel"]
  model_name = model_info["name"]
  config = model_info["config"]
  checkpoint = model_info["checkpoint"]

  sam = sam_model_registry[config](checkpoint=checkpoint)
  sam.to(device=device)
  predictor = SamPredictor(sam)
  print(f"======= Loaded model: {model_info['name']}")

  level_list=['1','2','3']
  complexity_list=['A','B','C']

  for level in level_list:
    for complexity in complexity_list:
      level_path='Level '+level+'/'+complexity+level
      tif_path=data_folder_path+level_path
      tif_list= [f[:-4] for f in os.listdir(tif_path) if f.endswith('.tif')]
      print(level_path)
      for uid in  tif_list:
        #if uid in valid_uid_list:
          target_uid=uid
          print(f"Processing {uid}")
          # ==============================================================================
          # Process Tiff
          # ==============================================================================
          geotiff_path = tif_path+'/'+uid+'.tif'
          base_tiff =uid+'.tif'
          start_time_utc = datetime.utcnow().isoformat()

          with rasterio.open(geotiff_path) as geotiff_reader:
            allbands_array = geotiff_reader.read()
            tiff_profile= geotiff_reader.profile.copy()

          tiff_image = np.transpose(allbands_array, (1, 2, 0))
          tiff_image_uint8 = tiff_image.astype(np.uint8)
          tiff_image_rgb=tiff_image_uint8[:, :, :3]

          # ==============================================================================
          # Process Geojson
          # ==============================================================================

          good_jsondf = gpd.read_file(good_geojson_path)
          good_json_uid_record = good_jsondf[good_jsondf['UID'] == uid]
          bad_jsondf = gpd.read_file(bad_geojson_path)
          bad_json_uid_record = bad_jsondf[bad_jsondf['UID'] == uid]

          if len(bad_json_uid_record) == 0:
            with open(missing_uid_list_path, "a") as f:
                f.write(f"{uid}  {level_path} \n")
            continue
          else:
            good_polygon_geom = good_json_uid_record.iloc[0].geometry
            ground_truth = rasterize(
                [(good_polygon_geom, 1)],
                out_shape=(geotiff_reader.height, geotiff_reader.width),
                transform=geotiff_reader.transform,
                fill=0,
                all_touched=True
            ).astype(bool)

            bad_polygon_geom = bad_json_uid_record.iloc[0].geometry
            bad_polygon_mask = rasterize(
              [(bad_polygon_geom, 1)],
              out_shape=(geotiff_reader.height, geotiff_reader.width),
              transform=geotiff_reader.transform,
              fill=0,
              all_touched=True
            )
            bad_polygon_mask_input = np.expand_dims(bad_polygon_mask, axis=0)
            scaled_bad_polygon_mask_input = np.where(bad_polygon_mask_input == 1, 5.0, -5.0).astype(np.float32)

            predictor.set_image(tiff_image_rgb)
            masks, scores, logits = predictor.predict(
              mask_input=scaled_bad_polygon_mask_input,
              multimask_output=True,
            )

            for i, mask in enumerate(masks):
              mask_bool=mask.astype(bool)
              intersection_mask = ground_truth & mask_bool
              union_mask = ground_truth | mask_bool
              iou = intersection_mask.sum() / union_mask.sum()
              iou_val=float(round(iou, 2))
              score_val = "{:.2f}".format(scores[i])
              rgb_chw = np.transpose(tiff_image_rgb, (2, 0, 1))
              mask_chw = mask[None, :, :]
              output_array = np.vstack([rgb_chw, mask_chw])
              tiff_profile.update(count=4, dtype=rasterio.uint8)
              tiff_profile.pop("nodata", None)
              output_tif_fileId = f'{target_uid}_{model_name}_{prompt}_prompt_mask_{i}'
              output_tif_filename = output_tif_fileId+'.tif'
              output_path = output_images_folder_path+output_tif_filename

            # ==============================================================================
            # Create Png
            # ==============================================================================

              png_path = output_images_folder_path+output_tif_fileId+'.png'
              mask = (mask > 0).astype(np.uint8)
              rgba_image = np.zeros((tiff_image_rgb.shape[0], tiff_image_rgb.shape[1], 4), dtype=np.uint8)
              rgba_image[:, :, :3] = tiff_image_rgb
              rgba_image[:, :, 3] = 255
              overlay = np.zeros_like(rgba_image)
              overlay[:, :, 0] = mask * 255
              overlay[:, :, 3] = mask * int(255 * 0.3)
              base = Image.fromarray(rgba_image)
              mask_overlay = Image.fromarray(overlay)
              composited = Image.alpha_composite(base, mask_overlay)
              composited.save(png_path)

              # ==============================================================================
              # Create Prediction geojson
              # ==============================================================================
              mask_polygons = [shape(geom) for geom, value in shapes(mask_bool.astype(np.uint8), mask=mask_bool, transform=geotiff_reader.transform) if value == 1]

              if len(mask_polygons) == 1:
                combined_geom = mask_polygons[0]
              else:
                combined_geom = MultiPolygon(mask_polygons)

              record = {
                "geometry": combined_geom,
                "UID": target_uid,
                "output_fileid": output_tif_fileId,
                "basemodel": basemodel,
                "model_name": model_name,
                "prompt": prompt,
                "iou": iou_val,
                "maskid": i
              }
              polygon_records.append(record)

              with rasterio.open(output_path, 'w', **tiff_profile) as dst:
                dst.write(output_array)

              end_time_utc = datetime.utcnow().isoformat()

              complexity_level=complexity+level
              rts_df = pd.DataFrame([{
                'uid': target_uid,
                'labeling_status': 'Auto_Generated',
                'approval_status': 'Pending',
                'base_tiff': base_tiff,
                'level': level,
                'complexity': complexity_level,
                'base_model': basemodel,
                'model_name': model_name,
                'prompt': prompt,
                'mask id': i,
                'iou':  iou_val,
                'sam score':score_val,
                'output_fileid': output_tif_fileId,
                'output_filename': output_tif_filename,
                'output_folder_location': output_folder_path,
                'start_time_utc': start_time_utc,
                'end_time_utc': end_time_utc,
                'notes':''
              }])

              if os.path.exists(manifest_path):
                rts_df.to_csv(manifest_path, mode='a', header=False, index=False)
              else:
                rts_df.to_csv(manifest_path, index=False)
if polygon_records:
  gdf = gpd.GeoDataFrame(polygon_records, crs="EPSG:3413")
  gdf.to_file(geojson_output_path, driver="GeoJSON")
  print(f"SAM output polygons saved to: {geojson_output_path}")
else:
  print("No mask polygons were generated to save.")

print("Processing Completed")

======= Loaded model: sam_base
Level 1/A1
Processing b88297fa-643c-5a51-b74c-03a8ce4ea9f6
Processing 0d8eea0e-e5c6-5ea2-ad38-f2050a03c131
Processing 128e871a-c0b5-50c6-8ce3-328af34d6a4d
Processing 5151c4fb-57c4-57cb-89dc-51b751635153
Processing 0b619729-9304-5884-bf02-73c1fbe866f0
Processing 811a1621-d78c-5331-86dd-f9d9020548f5
Processing 1fbe7655-5779-516b-ad3e-5d7be9729da9
Processing d3937257-4852-59f5-a588-a9b2e34b0d24
Processing 7d5ef26a-03c7-535e-b1c8-1c62ea57bf85
Processing 0a7e324b-142d-5f3f-bf5f-6ed1a0205542
Processing 8f47ab65-bb3a-5af2-bd41-16de15b7812c


/tmp/ipython-input-3625359817.py:77: RuntimeWarning: invalid value encountered in cast
  tiff_image_uint8 = tiff_image.astype(np.uint8)


Processing 95d5860f-c615-5903-833c-bbcb2a70ab5e
Processing c63ba86d-2019-52ad-9ecf-99fdfd3df4e3
Processing 24575e45-044f-5927-81af-63d0b3239dd6
Processing 177c743c-fbb8-50e7-a209-4c9c4cdaf860
Processing 18191cf6-cdbc-5e6e-b28c-90222c259b3d


/tmp/ipython-input-3625359817.py:77: RuntimeWarning: invalid value encountered in cast
  tiff_image_uint8 = tiff_image.astype(np.uint8)


Level 1/B1
Processing f9411380-b8c4-52ac-941f-200b351096d2
Processing be059998-1d07-5948-8bf5-a457c669b662
Processing c5367225-6b58-5bdd-8bec-92a2e5bc16d2
Processing 4afc59cb-a892-5f48-a87d-3e0b5faa122b
Processing 67ad0917-30cc-524f-88dd-6793000f2365
Processing 1aa83b04-1661-5aa9-9660-365cbb220886
Processing d532083a-3ed6-5d31-bb3d-201766541bc2


...

Processing d1e32f69-5218-5fa2-8317-f52c7a07a3b0
Processing 4761fc67-35a3-55f7-9aa7-4257de9c093e
Processing 5a7a3005-ac88-5863-ae02-7c38e9b6d668
Processing 46e5de7c-23a9-5d52-a8c1-e637b05b9437
Processing 4dd85204-3842-5cff-8602-117a20216dbf
Processing 238d3e11-790e-578d-b7d9-9f11b56f5840
Processing d6966047-4a0a-5323-b072-04181ebcd098
Processing b11f6531-556d-5e23-9b4f-46bc12bd789e
Processing 1c5575fb-9115-5930-8197-3acbac140931


/tmp/ipython-input-3625359817.py:77: RuntimeWarning: invalid value encountered in cast
  tiff_image_uint8 = tiff_image.astype(np.uint8)


Processing b4c2433b-e9df-586e-b1f5-b154058419ac


/tmp/ipython-input-3625359817.py:77: RuntimeWarning: invalid value encountered in cast
  tiff_image_uint8 = tiff_image.astype(np.uint8)


Processing c19a784f-898f-50fb-b9e7-da61565586f6
Processing 7e70f595-932f-586a-871a-4b937b977548
Processing 5fd172f3-8f9e-59e4-8883-27848133473f
Processing d2422f90-a1a4-53ca-8c9c-abadccdd3713
Processing 0d4dafde-ced6-5037-8e12-413de28a6140
Processing c03deb60-953c-5d19-b2a2-ace4e5e1bb45
Processing 93c64ecf-db54-562b-89f7-b3babd77f4d5
Processing f9593874-e718-5b9f-a9ed-03d39c588686
Processing 4cd473b0-40d5-5df2-9204-5e58ec38f7d4
SAM output polygons saved to: /content/drive/MyDrive/img-label-correction-SAM/output/metadata/predicted_polygons_sam_mask_prompt.geojson
Processing Completed
